In [106]:
pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [107]:
import torch
import torchtext
import gradio as gr

In [108]:
glove = torchtext.vocab.GloVe(name="6B", dim=100)

In [109]:
class cnn(torch.nn.Module):
      def __init__(self, vocab, emdedding_size):
        super(cnn,self).__init__()
        self.embedding = torch.nn.Embedding.from_pretrained(vocab.vectors)
        self.cnn_layer1 = torch.nn.Conv2d(1,20,(2,100),1,bias=False)
        self.cnn_layer2 = torch.nn.Conv2d(1,20,(4,100),1,bias=False)
        self.activation = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
        self.linear = torch.nn.Linear(40,1)
        
      def forward(self, x):
        length = len(x)
        embedded_result = self.embedding(x)
        embedded_result = embedded_result.permute(1,0,2)
        embedded_result = embedded_result.unsqueeze(1)
        cnn_layer1_result = self.cnn_layer1(embedded_result)
        cnn_layer2_result = self.cnn_layer2(embedded_result)
        layer1_activation = self.activation(cnn_layer1_result)
        layer2_activation = self.activation(cnn_layer2_result)
        layer1_activation = layer1_activation.squeeze()
        layer2_activation = layer2_activation.squeeze()
        max_pool_layer1_result = torch.nn.functional.max_pool1d(layer1_activation,int(length-1))
        max_pool_layer2_result = torch.nn.functional.max_pool1d(layer2_activation,int(length-3))
        max_pool_layer1_result = max_pool_layer1_result.squeeze()
        max_pool_layer2_result = max_pool_layer2_result.squeeze()        
        concatenated_result = torch.cat((max_pool_layer1_result,max_pool_layer2_result))
        prediction = self.linear(concatenated_result)
        prediction = self.sigmoid(prediction)
        return prediction

In [110]:
class baselineModel(torch.nn.Module):
      def __init__(self, vocab, emdedding_size):
        super(baselineModel,self).__init__()
        self.embedding = torch.nn.Embedding.from_pretrained(vocab.vectors)
        self.linear = torch.nn.Linear(emdedding_size,1)
        self.sigmoid = torch.nn.Sigmoid()
      def forward(self, x):
        embedded_result = self.embedding(x)
        average_result = torch.mean(embedded_result,0)
        prediction = self.linear(average_result)
        prediction = self.sigmoid(prediction)
        return prediction


In [111]:
def classify(text):
  glove = torchtext.vocab.GloVe(name="6B", dim=100)
  checkpoint_baseline = torch.load("baseline.pt")
  baseline_model = baselineModel(glove,100)
  baseline_model.load_state_dict(checkpoint_baseline)

  checkpoint_cnn = torch.load("cnn.pt")
  cnn_model = cnn(glove,100)
  cnn_model.load_state_dict(checkpoint_cnn)

  tokens = text.split()
  token_ints = [glove.stoi.get(tok, len(glove.stoi)-1) for tok in tokens]
  token_tensor = torch.LongTensor(token_ints).view(-1,1)

  baselinePrediction = baseline_model(token_tensor)
  baselineProb = float(baselinePrediction)
  
  cnnPrediction = cnn_model(token_tensor)
  cnnProb = float(cnnPrediction)


  if baselineProb > 0.5:
        a,b = "subjective", baselineProb
  else:
       a,b = "objective", baselineProb

  if cnnProb > 0.5:
        c,d = "subjective", cnnProb
  else:
       c,d = "objective", cnnProb
  return a,b,c,d

output1 = gr.Textbox(label="Baseline classification")
output2 = gr.Textbox(label="Baseline prediction probability")

output3 = gr.Textbox(label="CNN classification")
output4 = gr.Textbox(label="CNN prediction probability")

demo = gr.Interface(fn=classify, inputs="text", outputs=[output1, output2, output3, output4])
demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://29585.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f79bda5ae90>,
 'http://127.0.0.1:7861/',
 'https://29585.gradio.app')